## Joining parking ticket data to CentreLines for future mapping

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import zipfile

In [19]:
dfs = gpd.read_file('data/centreline/Centreline - Version 2.geojson')

In [21]:
dfs = dfs[["CENTRELINE_ID","LINEAR_NAME_FULL","LINEAR_NAME_FULL_LEGAL","LO_NUM_L","HI_NUM_L","LO_NUM_R","HI_NUM_R","PARITY_L","PARITY_R"]]
dfs["LINEAR_NAME_FULL"] = dfs["LINEAR_NAME_FULL"].str.upper()
dfs["LINEAR_NAME_FULL_LEGAL"] = dfs["LINEAR_NAME_FULL_LEGAL"].str.upper()
dfs = pd.DataFrame(dfs.loc[(dfs["PARITY_L"] != "N") | (dfs["PARITY_R"] != "N")])

In [22]:
dfs

,CENTRELINE_ID,LINEAR_NAME_FULL,LINEAR_NAME_FULL_LEGAL,LO_NUM_L,HI_NUM_L,LO_NUM_R,HI_NUM_R,PARITY_L,PARITY_R
1,914601,TWELFTH ST,TWELFTH STREET,66.0,92.0,65.0,89.0,E,O
2,7862398,THIRTEENTH ST,THIRTEENTH STREET,66.0,96.0,65.0,91.0,E,O
3,914587,LAKE SHORE BLVD W,LAKE SHORE BOULEVARD WEST,NaN,NaN,3180.0,3180.0,N,E
4,6735911,LAKE SHORE BLVD W,LAKE SHORE BOULEVARD WEST,3197.0,3197.0,3190.0,3190.0,O,E
5,20145557,CRESCENT RD,CRESCENT ROAD,146.0,146.0,NaN,NaN,E,N
...,...,...,...,...,...,...,...,...,...
70855,30021984,MONTCLAIR AVE,MONTCLAIR AVENUE,55.0,91.0,60.0,80.0,O,E
70856,30006687,QUEEN'S DR,QUEEN'S DRIVE,600.0,610.0,599.0,613.0,E,O
70858,8106186,RATHBURN RD,RATHBURN ROAD,367.0,397.0,372.0,400.0,O,E
70859,910937,THE EAST MALL,THE EAST MALL,596.0,596.0,NaN,NaN,E,N


In [35]:
# function for input address and return CENTRELINE_ID

def linear_geocode(address):
    
    try:
        number = int(address.split(' ', 1)[0])
        street = address.split(' ', 1)[1]
        dfsf = dfs.loc[dfs['LINEAR_NAME_FULL'] == street]
        dfsf = dfsf.loc[((dfsf["LO_NUM_L"] <= number) & (dfsf["HI_NUM_L"] >= number)) | ((dfsf["LO_NUM_R"] <= number) & (dfsf["HI_NUM_R"] >= number))]
        
        if len(dfsf) == 1:
            centreline = int(dfsf["CENTRELINE_ID"])
            return centreline
            
        elif len(dfsf) > 1:

            maybe = int(dfsf.head(1)["CENTRELINE_ID"])
            
            if number % 2 == 0:
                dfsf = (dfsf.loc[(
                    ((dfsf["PARITY_L"] != "O") & ((dfsf["LO_NUM_L"] <= number) & (dfsf["HI_NUM_L"] >= number))) |
                    ((dfsf["PARITY_R"] != "O") & ((dfsf["LO_NUM_R"] <= number) & (dfsf["HI_NUM_R"] >= number)))
                )])
            if number % 2 == 1:
                dfsf = (dfsf.loc[(
                    ((dfsf["PARITY_L"] != "E") & ((dfsf["LO_NUM_L"] <= number) & (dfsf["HI_NUM_L"] >= number))) |
                    ((dfsf["PARITY_R"] != "E") & ((dfsf["LO_NUM_R"] <= number) & (dfsf["HI_NUM_R"] >= number)))
                )])
                
            if len(dfsf) == 1:
                centreline = int(dfsf["CENTRELINE_ID"])
            elif len(dfsf) > 1:
                centrerline = int(dfsf.head(1)["CENTRELINE_ID"])
            else:
                centrerline = maybe
            
            return centreline
                
        else:
            
            None
        
        
    except:
        None
        
        return 0
    
linear_geocode("4700 JANE ST")


439138

In [5]:
years = [2020]

for year in years:
    dfs = []
    zf = zipfile.ZipFile("data/parking-tickets/parking-tickets-" + str(year) + ".zip")
    csvs = zf.namelist()
    for csv in csvs:
        dfs.append(pd.read_csv(zf.open(csv), engine="python", on_bad_lines='skip'))
        print(csv)
    df = pd.concat(dfs)
    

Parking_Tags_Data_2020.000.csv
Parking_Tags_Data_2020.001.csv
Parking_Tags_Data_2020.002.csv
Parking_Tags_Data_2020.003.csv
Parking_Tags_Data_2020.004.csv
Parking_Tags_Data_2020.005.csv


In [30]:
df.head(100)

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***60020,20200101,3,PARK ON PRIVATE PROPERTY,30,1.0,NaN,4700 JANE ST,NaN,NaN,SK
1,***46473,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,OPP,573 CLENDENAN AVE,NaN,NaN,ON
2,***46474,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,2.0,NR,573 CLENDENAN AVE,NaN,NaN,ON
3,***46475,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,4.0,NR,21 MARIA ST,NaN,NaN,ON
4,***46674,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,4.0,NR,164 GLENHOLME AVE,NaN,NaN,ON
...,...,...,...,...,...,...,...,...,...,...,...
95,***12447,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,28.0,NR,515 NAIRN AVE,NaN,NaN,ON
96,***46720,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,28.0,NR,289 WINDERMERE AVE,NaN,NaN,ON
97,***71465,20200101,29,PARK PROHIBITED TIME NO PERMIT,30,28.0,E/S,CRAWFORD ST,S/O,SULLY CRES,ON
98,***95016,20200101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,50,29.0,NR,40 RICHVIEW RD,NaN,NaN,ON


In [37]:
p,f

(805, 195)

In [34]:
p = 0
f = 0
for index, row in df.head(1000).iterrows():
    cl = linear_geocode(row['location2'])
    if cl == 0:
        f += 1
        print(row["location2"],row["location3"],row["location4"])
    else:
        p += 1

meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
GILMOUR AVE
GILMOUR AVE S/O MARIA ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
GILMOUR AVE
GILMOUR AVE S/O MARIA ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
SHAW ST
SHAW ST S/O COLLEGE ST
meow
CLENDENAN AVE
CLENDENAN AVE S/O ANNETTE ST
meow
HARRISON ST
HARRISON ST E/O SHAW ST
moo
30 TEESDAE PL
30 TEESDAE PL nan nan
meow
CRAWFORD ST
CRAWFORD ST N/O HARRISON ST
meow
CRAWFORD ST
CRAWFORD ST S/O SULLY CRES
meow
CRAWFORD ST
CRAWFORD ST S/O SULLY CRES
meow
CRAWFORD ST
CRAWFORD ST S/O SULLY CRE